This example notebook shows how to use the cosine similarity kernel for some common tasks. In this case we consider [current home-page example](https://matchms.readthedocs.io/en/latest/#example) of pesticide similarity evaluation.

In [23]:
%load_ext autoreload
%autoreload 2
from nbutils import chdir_to_root
chdir_to_root()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
from matchms.importing import load_from_mgf
from matchms.filtering import default_filters
from matchms.filtering import normalize_intensities
from matchms import calculate_scores
from matchms.similarity import CosineGreedy

from cudams.similarity import CudaCosineGreedy

In [22]:
file = list(load_from_mgf('data/pesticides.mgf'))
# Apply filters to clean and enhance each spectrum
spectrums = []
for spectrum in file:
    # Apply default filter to standardize ion mode, correct charge and more.
    # Default filter is fully explained at https://matchms.readthedocs.io/en/latest/api/matchms.filtering.html .
    spectrum = default_filters(spectrum)
    # Scale peak intensities to maximum of 1
    spectrum = normalize_intensities(spectrum)
    spectrums.append(spectrum)

In [ ]:
from cudams import 

In [ ]:
# Calculate Cosine similarity scores between all spectrums
# For other similarity score methods see https://matchms.readthedocs.io/en/latest/api/matchms.similarity.html .
# Because references and queries are here the same spectra, we can set is_symmetric=True
scores = calculate_scores(references=spectrums,
                          queries=spectrums,
                          similarity_function=CosineGreedy(),
                          is_symmetric=True)

In [ ]:

# This computed all-vs-all similarity scores, the array of which can be accessed as scores.scores
print(f"Size of matrix of computed similarities: {scores.scores.shape}")

# Matchms allows to get the best matches for any query using scores_by_query
query = spectrums[15]  # just an example
best_matches = scores.scores_by_query(query, 'CosineGreedy_score', sort=True)

# Print the calculated scores for each spectrum pair
for (reference, score) in best_matches[:10]:
    # Ignore scores between same spectrum
    if reference is not query:
        print(f"Reference scan id: {reference.metadata['scans']}")
        print(f"Query scan id: {query.metadata['scans']}")
        print(f"Score: {score[0]:.4f}")
        print(f"Number of matching peaks: {score[1]}")
        print("----------------------------")
